# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `01_materials/labs/2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [13]:
# Write your code below.
%reload_ext dotenv
%dotenv 

In [ ]:
import dask.dataframe as dd


+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [ ]:
import os
from glob import glob


PRICE_DATA = os.environ.get("PRICE_DATA")
print(PRICE_DATA)


parquet_files = glob(os.path.join(PRICE_DATA, "**/*.parquet"), recursive=True)
parquet_files
dd_px = dd.read_parquet(parquet_files).set_index("ticker")



../../05_src/data/prices/


,Date,Open,High,Low,Close,Adj Close,Volume,source,Year
ticker,,,,,,,,,
ACN,2001-07-19,15.10,15.29,15.00,15.17,11.404394,34994300.0,ACN.csv,2001
ACN,2001-07-20,15.05,15.05,14.80,15.01,11.284108,9238500.0,ACN.csv,2001
ACN,2001-07-23,15.00,15.01,14.55,15.00,11.276587,7501000.0,ACN.csv,2001
ACN,2001-07-24,14.95,14.97,14.70,14.86,11.171341,3537300.0,ACN.csv,2001
ACN,2001-07-25,14.70,14.95,14.65,14.95,11.238999,4208100.0,ACN.csv,2001


For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Close:
    
    - `returns`: (Close / Close_lag_1) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [23]:
import pandas as pd

dd_shift = (
    dd_px
        .groupby('ticker', group_keys=False)
        .apply(
            lambda x: x.sort_values('Date', ascending = True)
                       .assign(Close_lag_1 = x['Close'].shift(1), Adj_Close_lag_1 = x['Adj Close'].shift(1)), 

            meta = pd.DataFrame(data ={'Date': 'datetime64[ns]',
                    'Open': 'f8',
                    'High': 'f8',
                    'Low': 'f8',
                    'Close': 'f8',
                    'Adj Close': 'f8',
                    'Volume': 'i8',
                    'source': 'object',
                    'Year': 'int32',
                    'Close_lag_1': 'f8',
                    'Adj_Close_lag_1': 'f8' },
                    index = pd.Index([], dtype=pd.StringDtype(), name='ticker'))
    ))

dd_feat = dd_shift.assign(
    Returns = lambda x: x['Close']/x['Close_lag_1'] - 1,
    hi_lo_range = lambda x: x['High'] - x['Low']
)

dd_feat.head()


,Date,Open,High,Low,Close,Adj Close,Volume,source,Year,Close_lag_1,Adj_Close_lag_1,Returns,hi_lo_range
ticker,,,,,,,,,,,,,
ACN,2001-07-19,15.10,15.29,15.00,15.17,11.404394,34994300.0,ACN.csv,2001,NaN,NaN,NaN,0.29
ACN,2001-07-20,15.05,15.05,14.80,15.01,11.284108,9238500.0,ACN.csv,2001,15.17,11.404394,-0.010547,0.25
ACN,2001-07-23,15.00,15.01,14.55,15.00,11.276587,7501000.0,ACN.csv,2001,15.01,11.284108,-0.000666,0.46
ACN,2001-07-24,14.95,14.97,14.70,14.86,11.171341,3537300.0,ACN.csv,2001,15.00,11.276587,-0.009333,0.27
ACN,2001-07-25,14.70,14.95,14.65,14.95,11.238999,4208100.0,ACN.csv,2001,14.86,11.171341,0.006057,0.30


+ Convert the Dask data frame to a pandas data frame. 
+ Add a new feature containing the moving average of `returns` using a window of 10 days. There are several ways to solve this task, a simple one uses `.rolling(10).mean()`.

(3 pt)

In [24]:

# Convert Dask -> pandas
df_feat = dd_feat.compute()
print(type(df_feat))

# Ensure sorted for group-wise rolling
df_feat = df_feat.sort_values(['ticker', 'Date'])

# 10-day moving average of returns, per ticker
df_feat['ma10_returns'] = (
    df_feat
      .groupby('ticker', group_keys=False)['Returns']
      .rolling(10)
      .mean()
      .reset_index(level=0, drop=True)
)

df_feat.head()




<class 'pandas.core.frame.DataFrame'>


,Date,Open,High,Low,Close,Adj Close,Volume,source,Year,Close_lag_1,Adj_Close_lag_1,Returns,hi_lo_range,ma10_returns
ticker,,,,,,,,,,,,,,
ACN,2001-07-19,15.10,15.29,15.00,15.17,11.404394,34994300.0,ACN.csv,2001,NaN,NaN,NaN,0.29,NaN
ACN,2001-07-20,15.05,15.05,14.80,15.01,11.284108,9238500.0,ACN.csv,2001,190.970001,189.420700,-0.921401,0.25,NaN
ACN,2001-07-23,15.00,15.01,14.55,15.00,11.276587,7501000.0,ACN.csv,2001,191.389999,189.837296,-0.921626,0.46,NaN
ACN,2001-07-24,14.95,14.97,14.70,14.86,11.171341,3537300.0,ACN.csv,2001,190.309998,188.766068,-0.921917,0.27,NaN
ACN,2001-07-25,14.70,14.95,14.65,14.95,11.238999,4208100.0,ACN.csv,2001,191.089996,189.539734,-0.921765,0.30,NaN


Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?

Answer: No. Dask can do the calculation as well.

+ Would it have been better to do it in Dask? Why?

Answer: No, it is better to do it in Panda, because the dataset is small and can be fitted into memory. Using Dask, you will need to go across different partitions and more works.

(1 pt)

## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.